# Interview Project

In this notebook, I explore the data provided for question #1 and #2, and demonstrate my data exploration methods and reasoning to identify potential markets.

## 1.) Import Libraries and Set Options

In [ ]:
import pandas as pd
import altair as alt
from sklearn.cluster import DBSCAN

In [ ]:
# Altair is a visualization tool similar to Matplotlib, Seaborn, or other tools. Altair has beautiful themese that you can use, so here I selected the 'fiverthirtyeight' theme, which replicates the visual style that fivethirtyeight typically uses.
# Altair also imposes a limit on 4,000 rows of data by default, so I set alt.data_transformers.disable_max_rows() to display certain visualizations.
alt.themes.enable('fivethirtyeight')
alt.data_transformers.disable_max_rows()

## 2.) Load and Process Data

In [ ]:
# load the PLD for Q1
q1_df = pd.read_csv('data/shipments/pld_table/202302 BIE Technical Interview PLD1.csv')

In [ ]:
# Initial look at the data
q1_df.head()

In [ ]:
# view the shape of the data
q1_df.shape

In [ ]:
# view datatypes
q1_df.dtypes

In [ ]:
# convert origin and destination zip codes to string type so Altair recognizes it as categorical instead of numerical. This can sometimes effect visualizations, and zip codes should be categorical data anyway.
q1_df['OriginZip'] = q1_df['OriginZip'].astype(str)
q1_df['DestinationZip'] = q1_df['DestinationZip'].astype(str)

In [ ]:
# looks like leading 0s were dropped from zip codes at some point in exporting or loading the csv, let's fill in these leading 0s. Some are missing 2 leadings 0s and others only one.

print(len(q1_df[q1_df['DestinationZip'].str.len() == 3]), len(q1_df[q1_df['DestinationZip'].str.len() == 4]))

In [ ]:
# create and appl function to fill in leading 0s
def fill_leading_zeros(x):
    x = x.zfill(5)
    return x

q1_df['OriginZip'] = q1_df['OriginZip'].apply(fill_leading_zeros)
q1_df['DestinationZip'] = q1_df['DestinationZip'].apply(fill_leading_zeros)

In [ ]:
# check if any zip codes are missing leading 0s again
print(len(q1_df[q1_df['DestinationZip'].str.len() == 3]), len(q1_df[q1_df['DestinationZip'].str.len() == 4]))

## 3.) Visualize Data

In [ ]:
# view the distribution of origin zip codes
origin_zips = q1_df['OriginZip'].value_counts(ascending=False).reset_index()

In [ ]:
origin_zips

In [ ]:
# It looks like origin zip codes are heavily skewed, a lot of volume is originating from the same place

In [ ]:
# Here we will plot an altair bar chart to show the distribution of origin zip codes. The visualization includes a tooltip that you can hover over to view the zip code and count for each card. You can also pass in the original dataframe if you want to view more tooltips such as destination zip, total order count, average package weight, etc.
alt.Chart(origin_zips).mark_bar().encode(
    x=alt.X('OriginZip:N', axis=alt.Axis(title='Origin Zip Code'), sort=None),
    y=alt.Y('count:Q', axis=alt.Axis(title='Count')),
    tooltip=['OriginZip', 'count']
).configure_axisX(
    labels=False
).properties(
    width=2500,
    height=500,
    title='Distribution of Origin Zip Codes'
).configure_title(
    anchor='middle'
)

In [ ]:
# view the distribution of destination zip codes.
destination_zips = q1_df['DestinationZip'].value_counts(ascending=False).reset_index()

In [ ]:
destination_zips

In [ ]:
# Destination zip codes appear to follow a similar distirbution to origin zip codes, although there are significantly more destinations than origins.

In [ ]:
# plot distribution of destination zip codes with tool tips. Since there are so many data points, it is difficult to see. This could be improved by filtering for a specific origin zip code.
alt.Chart(destination_zips).mark_bar().encode(
    x=alt.X('DestinationZip:N', axis=alt.Axis(title='Destination Zip Code'), sort=None),
    y=alt.Y('count:Q', axis=alt.Axis(title='Count')),
    tooltip=['DestinationZip', 'count']
).configure_axisX(
    labels=False
).properties(
    width=2500,
    height=500,
    title='Distribution of Destination Zip Codes'
).configure_title(
    anchor='middle'
)

In [ ]:
# group by origin an destination zip and take sum of total packages for a specific route
grouped_q1_df = q1_df.groupby(by=['OriginZip', 'DestinationZip']).sum().reset_index()

In [ ]:
grouped_q1_df

In [ ]:
# It looks like there are 323,938 distinct routes

In [ ]:
# here we will create a concatenated column which contains the origin and destination zip codes combined, which makes it easier to plot a specific "route".
grouped_q1_df['Route'] = 'Origin: ' + grouped_q1_df['OriginZip'] + '- Destination: ' + grouped_q1_df['DestinationZip']

In [ ]:
# since this is for September, 2022, just create a variable for # of days. This could be dynamic if we had more months, or likely would already have a feature denoting the month.
days_in_september = 30

In [ ]:
# calculate the average packages per day for a given route through the month of September.
grouped_q1_df['AveragePerDay'] = (grouped_q1_df['OrderCount'] / days_in_september).round().astype(int)

In [ ]:
# view the new dataframe in descending order by average packages per day.
grouped_q1_df.sort_values('AveragePerDay',ascending=False)

In [ ]:
# The average is rounded to the nearest number, some routes averaged less than 1 per day, and were therefore rounded down to 0. If a client wanted, we could represent this in a different way.

In [ ]:
# While the question asks about average packages per day, it is still important to view the distribution of the total number of orders.

In [ ]:
order_counts = grouped_q1_df.sort_values('OrderCount', ascending=False)

In [ ]:
alt.Chart(order_counts).mark_bar().encode(
    x=alt.X('Route:N', axis=alt.Axis(title='Route'), sort=None),
    y=alt.Y('OrderCount:Q', axis=alt.Axis(title='Order Count')),
    tooltip=['Route:N', 'OrderCount']
).configure_axisX(
    labels=False
).properties(
    width=1500,
    height=200,
    title='Distribution of Zip Code Routes by Order Count'
).configure_title(
    anchor='middle'
)

In [ ]:
average_order_counts = grouped_q1_df.sort_values('AveragePerDay', ascending=False)

In [ ]:
# here we see that some routes have an average of 0 packages per day. The average is rounded to the nearest whole number, but this can be updated if it is misleading.
average_order_counts

In [ ]:
# Here we will make a histogram, which is better to represent numerical data with a lot of values. The downside of the histogram is that you can't easily view tooltips.
alt.Chart(average_order_counts).mark_bar().encode(
    x=alt.X('AveragePerDay:Q', axis=alt.Axis(title='Average Order Count Per Day - Binned'), bin=True),
    y=alt.Y('count()', axis=alt.Axis(title='Number of Routes')),
).properties(
    width=1500,
    height=800,
    title='Distribution of Zip Code Routes by Average Daily Orders'
).configure_title(
    anchor='middle'
)

In [ ]:
# The distribution is heavily skewed. Most direct routes appear to have very little volume, although a route would have to handle at least 15 packages a month to reach an average of 1 per day.
# There are so many routes with an average of 0 per day, that the routes with a higher average are difficult to see. This makes this particular visualization misleading.

In [ ]:
# Let's look at the top route for a sanity check. It looks like one particular day they shipped 394 orders. Is there significance behind this day?
q1_df[(q1_df['OriginZip'] == '36039') & (q1_df['DestinationZip'] == '71322')]

## 4.) Question #2: Identify Potential Markets

In [ ]:
# now that we know what the volume looks like for given routes, let's look at destination zip codes again and get the total order count for a given destination
destination_monthly_total = grouped_q1_df.groupby('DestinationZip')['OrderCount'].sum().reset_index()
destination_monthly_total.sort_values('OrderCount', ascending=False)

In [ ]:
# We want to potential facilities in geographical areas where a large volume of packages are sent. In order to do this, we will use the sklearn DBSCAN algorithm, which stands for Density-Based Spatial Clustering Applications with Noise. 
# This finds clusters with high density and expands from there:  https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html
# In order to do this, we need to find the latitude and longitude coordinates of zip codes in the US. For this, I used this publicly available file found here https://www2.census.gov/geo/docs/maps-data/data/gazetteer/2020_Gazetteer/2020_Gaz_zcta_national.zip
# I am not able to verify if these are accurate, but even general accuracy should be good enough to form proper geographical clusters.

In [ ]:
# load in geographical data from the file
zip_code_df = pd.read_csv('data/2020_Gaz_zcta_national.txt', sep='\t')

In [ ]:
# GEO ID = zip code, and we are interested in INTPTLAT and INTPTLONG for the latitude and longtidue coordinates
zip_code_df

In [ ]:
# since these are also missing leadings 0s, let's apply them here
zip_code_df['DestinationZip'] = zip_code_df['GEOID'].astype(str).apply(fill_leading_zeros)

In [ ]:
zip_code_df.head()

In [ ]:
# join the monthly total count 
merged_df = pd.merge(destination_monthly_total, zip_code_df, on='DestinationZip', how='left')

In [ ]:

# view the columns. It looks like there's space characters after INTPTLONG, so let's replace those
merged_df.columns

In [ ]:
merged_df = merged_df.rename(columns=lambda x: x.replace(' ', ''))

In [ ]:
merged_df.columns

In [ ]:
# view the shape of the merged df
len(merged_df)

In [ ]:
# let's confirm if every zip code was matched
merged_df[merged_df['GEOID'].isna()].sort_values('OrderCount', ascending=False)

In [ ]:
# it looks like the provided zip code coordinates dataset may be incomplete.  It looks like we will lose about 4.627 rows, some of which have significant volume, which could greatly impact the results. 
# Neverthless, we will drop the NA values for the sake of this exercise and make a note to find a more accurate zip code geographical coordinate source in the future.
merged_df = merged_df.dropna()

In [ ]:
# create an input for DBSCAN
coords = merged_df[['INTPTLAT', 'INTPTLONG']].values

In [ ]:
coords

In [ ]:
# set dbscan hyperparameters eps=.5, min_samples=100. DBSCAN is highlysensitive to these values. These can be altered based on customer needs, as it will change the total number of clusters the algorithm identifies. These parameters create 8 clusters, which represent different geographical regions of the US.
dbscan_model = DBSCAN(eps=.5, min_samples=100)

In [ ]:
labels = dbscan_model.fit_predict(coords)

In [ ]:
merged_df['Cluster'] = labels

In [ ]:
merged_df.head()

In [ ]:
# quick view o the clusters and the total number of orders within them.
merged_df.groupby('Cluster').sum().sort_values('OrderCount', ascending=False)

In [ ]:
# Plot the clusters: -1 contains the noise points which the algorithm did not attribute to a cluster. In some cases, you want to remove these noise points when plotting, but as you can see, they create a nice shape of the United States, which serves as a visual aid in identifying potential markets.
alt.Chart(merged_df).mark_point(size=60).encode(
    x=alt.X('INTPTLONG', axis=alt.Axis(title='Longitude')),
    y=alt.Y('INTPTLAT', axis=alt.Axis(title='Latitude')),
    color=alt.Color('Cluster:N'),
    tooltip=['DestinationZip', 'OrderCount', 'Cluster']
).properties(
    height=800,
    width=1600,
    title='Potential Sortation Facilities via DBSCAN: eps=0.5, min_samples=100'
).configure_legend(
    padding=10,
    cornerRadius=10,
    orient='top-right'
).interactive()

## 5.) Question 3: Productionalizing Data

Below are the steps I would take and technologies I would use to productionalize different formats of PLDs from customers (as demosntrated in the file provided for question 3 located in /data/shipments/monthly_shipping_table//202302 BIE Technical Interview PLD2.csv), and turn them into visualizations or other tools for stakeholders to view results.

1. Extract:
    - Python offers a variety of packages for loading and processing data, which include pandas, PySpark, numpy, requests, SQLAlchemcy, and others, all of which can interact with data from different sources.
    - pandas is a quick and easy tool for data manipulation, however, as business needs scale and as data grows, PySpark is generally the better option. PySpark offers distributed processing which can speed up large data operations and allows data to be cached in memory, which improves performance.
    - All the examples I created above can be replicated above in PySpark on a larger scale and on multiple machines.
    - Python to validate data inputs against a schema (see example schema for provided datasets located in schemas/schemas.json) if required, as well is programatically accept and clean differnet filetypes, like JSON, XML, csv, Excel, etc.
    - In a traditional ELT Many of these steps are not necessary and Python can simply be used as a tool to retrieve raw data and load the raw data directly into a data warehouse like Snowflake or BigQuery. However, ELT can create higher loads and costs on the target system.

2. Transform:
    - There are a variety of tools available for this. The transformations could be done in PySpark to set up tables, version control
    - PySpark can be used to clean, aggregate, and enrich data as needed, and loaded into the target system.
    - Transformations are based on customer needs, and Pyspark can be version controlled.
    - If ETL is not necessary and the company could consider ELT, then a tool that I am partial to is dbt (data build tool), which can be used as a transformation tool on top of a data warehouse like Snowflake or BigQuery. dbt is a great tool because it's simple to use, it has built in testing an documentation, and is easy to integrate.
    - ELT vs. ETL ultimately depends on the needs of the company and the costs associated with it.

3. Load:
    - There are multiple options here depending on the needs of the business. A RDBMS might be required if there is transactional processing required to support daily operations of the organization, which then can be fed into a cloud-based data warehouse for analytical processes. (OLTP vs OLAP).
    - RDBMS can be useful if real-time reporting is required, but data warehouses are superior for analytical reporting as they can accept data from different sources.
    - Datawarehouses like Snowflake, BigQuery, or Redshift are extremely useful and be cost effective for analytical processing.

4. Visualiaztion
    - There are so many tools available for this, it's hard to pick one. Tableau, QlikView, Sisense, PowerBI, Looker, etc. This mostly just comes down to cost. Sisense is particularly good with embedding visualizations, but that is mostly useful for companies that offer a wide range of products and need to emebed visulizations within those products

4. Schedule:
    - Outside of just ETL, it is important to use some sort of data orchestration tool like Airflow or Argo Workflows (kubernetes) to schedule the pipeline to run at a specific time and monitor it for any issues.
    - Monitoring can alert teams if there are issues with the pipeline like data ingestion or data validation failures.




 Databricks is another useful tool that natively includes Apache Spark, but I do not know the needs of the company to determine if Databricks is the correct solution based on existing architecture. Databricks is a unfifed data analytics platform that you can use to handle the entire ETL pipeline and orchestrate tasks (although not as comprehnsive as Airflow). It all ultimately depends on the needs of the company. The above steps are a high level view of the steps I would take, but the technologies used for those steps can be substituted if needed.


You can view a sample flowchart of an ETL pipeline orchestrated using Airflow in images/ETL pipeline.png